# Deploy Python Function that includes custom .py

This notebook tested this [sample](https://github.com/IBMDataScience/sample-notebooks/blob/master/CloudPakForData/notebooks/Use_Python_function_feature_to_scrape_a__ndTSmTZh9.ipynb) to work in CP4D v4.x.

## 1. Define the function to be deployed: 

In [1]:
# You can add any information needed to run the Python function, e.g., wml credentials.
py_params = {

}

In [2]:
def py_funct(params=py_params):  
    try:
        # Import the subprocess module.
        import subprocess
        
        # Install required packages.
        subprocess.check_output('pip install --user lxml', stderr=subprocess.STDOUT, shell=True)
        subprocess.check_output('pip install --user bs4', stderr=subprocess.STDOUT, shell=True)
        subprocess.check_output('pip install --user sklearn', stderr=subprocess.STDOUT, shell=True)
    except subprocess.CalledProcessError as e:        
        install_err = 'subprocess.CalledProcessError:\n\n' + 'cmd:\n' + e.cmd + '\n\noutput:\n' + e.output.decode()
        raise Exception( 'Installation failed:\n' + install_err )
    
    def score(payload):
        try:
            # Import required modules.
            from bs4 import BeautifulSoup
            from urllib.request import urlopen
            from sklearn.feature_extraction.text import CountVectorizer

            urls = payload['input_data'][0]['values']
            final_texts = []   # An array that will have stripped clean text from html tag enclosed text.

            for url in urls:            
                html = urlopen(url)
                soup = BeautifulSoup(html, 'lxml')

                p_tags = soup.find_all('p')    # Text is enclosed in <p> tag.

                for p in p_tags:
                    str_p = str(p)
                    text = BeautifulSoup(str_p, 'lxml').get_text()
                    final_texts.append(text)

            vectorizer = CountVectorizer()
            vectorizer.fit_transform(final_texts)

            return {"predictions": [{"fields": ["tokens"], "values": [vectorizer.get_feature_names()]}]}
        except Exception as e:
            return {"predictions": [{"error" : repr(e)}]}
        
    return score

## 2. Define which deployment space you want to save this function to.

In [3]:
from ibm_watson_machine_learning import APIClient

In [4]:
# get the Project ID and set the location to save the model to the project
from ibm_watson_machine_learning import APIClient
import os

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": os.environ['RUNTIME_ENV_APSX_URL'],
   "version": "4.0"
}

client = APIClient(wml_credentials)

In [5]:
# Enter the name of your deployment space here:
space_uid = "a297dc8b-3ad4-4485-b67a-548b79a59ec8"
print("Space UID = " + space_uid)

Space UID = a297dc8b-3ad4-4485-b67a-548b79a59ec8


In [6]:
client.set.default_space(space_uid)

'SUCCESS'

## 3. Define Function metadata

In [11]:
# Function Metadata.
meta_props = {
    client.repository.FunctionMetaNames.NAME: "Python function demo",
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID:client.software_specifications.get_id_by_name("default_py3.7_opence"),
}

In [12]:
# if you don't know what software_spec are available, run the following line to check
#client.software_specifications.list()

## 4. Store function

In [13]:
#Create the function artifact.
function_artifact = client.repository.store_function(meta_props=meta_props, function=py_funct)
function_uid = client.repository.get_function_uid(function_artifact)
print("Function UID = " + function_uid)

Function UID = fd93bceb-d5ee-4430-8e55-ad62dd833c7c


In [14]:
# Details about the function.
function_details = client.repository.get_details(function_uid)
from pprint import pprint
pprint(function_details)

{'entity': {'software_spec': {'id': 'c2057dd4-f42c-5f77-a02f-72bdbd3282c9',
                              'name': 'default_py3.7_opence'},
            'type': 'python'},
 'metadata': {'created_at': '2022-04-19T20:07:52.398Z',
              'id': 'fd93bceb-d5ee-4430-8e55-ad62dd833c7c',
              'modified_at': '2022-04-19T20:07:53.680Z',
              'name': 'Python function demo',
              'owner': '1000331005',
              'space_id': 'a297dc8b-3ad4-4485-b67a-548b79a59ec8'},
 'system': {'warnings': []}}


In [15]:
# Display a list of all the functions.
client.repository.list_functions()

------------------------------------  --------------------  ------------------------  ------
GUID                                  NAME                  CREATED                   TYPE
fd93bceb-d5ee-4430-8e55-ad62dd833c7c  Python function demo  2022-04-19T20:07:52.002Z  python
------------------------------------  --------------------  ------------------------  ------


## 5. Define Deployment metdadata and Deploy Function

## 5.1 Online deployment

In [16]:
# Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "Web scraping python function deployment online",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [17]:
# Create the deployment.
deployment_details = client.deployments.create(function_uid, meta_props=deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: 'fd93bceb-d5ee-4430-8e55-ad62dd833c7c' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
...
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4ebc7c8e-e49f-40cc-a3a7-72f95161c26b'
------------------------------------------------------------------------------------------------




In [18]:
deployment_details

{'entity': {'asset': {'id': 'fd93bceb-d5ee-4430-8e55-ad62dd833c7c'},
  'custom': {},
  'deployed_asset_type': 'function',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'XXS', 'num_nodes': 1},
  'name': 'Web scraping python function deployment online',
  'online': {},
  'space_id': 'a297dc8b-3ad4-4485-b67a-548b79a59ec8',
  'status': {'online_url': {'url': 'https://internal-nginx-svc:12443/ml/v4/deployments/4ebc7c8e-e49f-40cc-a3a7-72f95161c26b/predictions'},
   'serving_urls': ['https://internal-nginx-svc:12443/ml/v4/deployments/4ebc7c8e-e49f-40cc-a3a7-72f95161c26b/predictions'],
   'state': 'ready'}},
 'metadata': {'created_at': '2022-04-19T20:08:18.036Z',
  'id': '4ebc7c8e-e49f-40cc-a3a7-72f95161c26b',
  'modified_at': '2022-04-19T20:08:18.036Z',
  'name': 'Web scraping python function deployment online',
  'owner': '1000331005',
  'space_id': 'a297dc8b-3ad4-4485-b67a-548b79a59ec8'},
 'system': {'warnings': [{'id': 'Deprecated',
    'message': 'online_url is deprecated and will b

### Test online endpoint

In [19]:
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment uid = {}'.format(deployment_uid))

Deployment uid = 4ebc7c8e-e49f-40cc-a3a7-72f95161c26b


In [20]:
# Prepare scoring payload for online scoring
job_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': ['url'],
        'values': [
            'https://www.ibm.com/cloud/machine-learning'
        ]
    }]
}
pprint(job_payload)

{'input_data': [{'fields': ['url'],
                 'values': ['https://www.ibm.com/cloud/machine-learning']}]}


In [21]:
job_payload['input_data'][0]['values'][0]

'https://www.ibm.com/cloud/machine-learning'

In [22]:
# if using UI for testing, use the following to get right format
#import json
#json.dumps(job_payload)

In [23]:
# Perform prediction and display the result.
# for online scoring
job_details = client.deployments.score(deployment_uid, job_payload)
pprint(job_details['predictions'][0]['values'][0][:10])

['01',
 '20',
 '2020',
 '37',
 'aberdeen',
 'ability',
 'about',
 'access',
 'accuracy',
 'across']


## 5.2 Batch deployment: update python function

In [24]:
def py_funct(params=py_params):  
    try:
        # Import the subprocess module.
        import subprocess
        
        # Install required packages.
        subprocess.check_output('pip install --user lxml', stderr=subprocess.STDOUT, shell=True)
        subprocess.check_output('pip install --user bs4', stderr=subprocess.STDOUT, shell=True)
        subprocess.check_output('pip install --user sklearn', stderr=subprocess.STDOUT, shell=True)
    except subprocess.CalledProcessError as e:        
        install_err = 'subprocess.CalledProcessError:\n\n' + 'cmd:\n' + e.cmd + '\n\noutput:\n' + e.output.decode()
        raise Exception( 'Installation failed:\n' + install_err )
    
    def score(payload):
        try:
            # Import required modules.
            from bs4 import BeautifulSoup
            from urllib.request import urlopen
            from sklearn.feature_extraction.text import CountVectorizer

            urls = payload['input_data'][0]['values']

            output = []
            for url in urls:
                final_texts = []
                html = urlopen(url[0])
                soup = BeautifulSoup(html, 'lxml')

                p_tags = soup.find_all('p')    # Text is enclosed in <p> tag.

                for p in p_tags:
                    str_p = str(p)
                    text = BeautifulSoup(str_p, 'lxml').get_text()
                    final_texts.append(text)
                vectorizer = CountVectorizer()
                vectorizer.fit_transform(final_texts)
                output.append(vectorizer.get_feature_names())

            return {"predictions": [{"fields": ["tokens"], "values": output}]}
        except Exception as e:
            return {"predictions": [{"error" : repr(e)}]}
        
    return score

In [25]:
# Function Metadata.
meta_props = {
    client.repository.FunctionMetaNames.NAME: "Web scraping python function updated for batch",
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID:client.software_specifications.get_id_by_name("default_py3.7"),
}

In [26]:
function_artifact = client.repository.store_function(meta_props=meta_props, function=py_funct)
function_uid = client.repository.get_function_uid(function_artifact)
print("Function UID = " + function_uid)

Failure during saving function. (POST https://internal-nginx-svc:12443/ml/v4/functions?version=2021-06-24)
Status code: 400, body: {
  "trace": "unknown",
  "errors": [{
    "code": "invalid_request_entity",
    "message": "Invalid request entity: Unsupported software specification provided in the request.",
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}


ApiRequestFailure: Failure during saving function. (POST https://internal-nginx-svc:12443/ml/v4/functions?version=2021-06-24)
Status code: 400, body: {
  "trace": "unknown",
  "errors": [{
    "code": "invalid_request_entity",
    "message": "Invalid request entity: Unsupported software specification provided in the request.",
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}

In [27]:
function_details = client.repository.get_details(function_uid)
from pprint import pprint
pprint(function_details)

{'entity': {'software_spec': {'id': 'c2057dd4-f42c-5f77-a02f-72bdbd3282c9',
                              'name': 'default_py3.7_opence'},
            'type': 'python'},
 'metadata': {'created_at': '2022-04-19T20:07:52.398Z',
              'id': 'fd93bceb-d5ee-4430-8e55-ad62dd833c7c',
              'modified_at': '2022-04-19T20:07:53.680Z',
              'name': 'Python function demo',
              'owner': '1000331005',
              'space_id': 'a297dc8b-3ad4-4485-b67a-548b79a59ec8'},
 'system': {'warnings': []}}


In [28]:
# Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "Web scraping python function deployment batch",
    #client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.BATCH: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name": "S", "num_nodes": 1}
}

In [29]:
#client.hardware_specifications.list()

In [30]:
# Create the deployment.
deployment_details = client.deployments.create(function_uid, meta_props=deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: 'fd93bceb-d5ee-4430-8e55-ad62dd833c7c' started

#######################################################################################


ready.


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4d310755-e047-4285-b759-91e66ac40fe8'
------------------------------------------------------------------------------------------------




In [31]:
deployment_details

{'entity': {'asset': {'id': 'fd93bceb-d5ee-4430-8e55-ad62dd833c7c'},
  'batch': {},
  'custom': {},
  'deployed_asset_type': 'function',
  'hardware_spec': {'id': 'e7ed1d6c-2e89-42d7-aed5-863b972c1d2b',
   'name': 'S',
   'num_nodes': 1},
  'name': 'Web scraping python function deployment batch',
  'space_id': 'a297dc8b-3ad4-4485-b67a-548b79a59ec8',
  'status': {'state': 'ready'}},
 'metadata': {'created_at': '2022-04-19T20:08:49.873Z',
  'id': '4d310755-e047-4285-b759-91e66ac40fe8',
  'modified_at': '2022-04-19T20:08:49.873Z',
  'name': 'Web scraping python function deployment batch',
  'owner': '1000331005',
  'space_id': 'a297dc8b-3ad4-4485-b67a-548b79a59ec8'}}

In [32]:
client.deployments.list()

------------------------------------  ----------------------------------------------  -----  ------------------------
GUID                                  NAME                                            STATE  CREATED
4d310755-e047-4285-b759-91e66ac40fe8  Web scraping python function deployment batch   ready  2022-04-19T20:08:49.873Z
4ebc7c8e-e49f-40cc-a3a7-72f95161c26b  Web scraping python function deployment online  ready  2022-04-19T20:08:18.036Z
a443e90e-8fdc-42bc-9d68-8356075d4aea  Churn Deployment via API-Online                 ready  2022-04-06T17:45:04.645Z
98744498-f1f5-455d-8c44-4306830234c0  Demo online                                     ready  2022-03-16T20:56:30.856Z
ab070bba-9a71-49eb-b8a2-4c9895fa7a87  Customer_churn_batch                            ready  2022-03-03T20:21:52.805Z
572b85c0-c994-4946-a5ac-9e77af3bf925  Customer_Churn_Online                           ready  2022-03-03T20:21:28.135Z
------------------------------------  -----------------------------------

### Test Batch scoring endpoint

In [157]:
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment uid = {}'.format(deployment_uid))

Deployment uid = 4b9d1226-d0e9-4c14-abfe-f5de97d409eb


In [158]:
job_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': ['url'],
        'values': [
            ['https://www.ibm.com/cloud/machine-learning'], ['https://en.wikipedia.org/wiki/IBM']
        ]
    }]
}
pprint(job_payload)

{'input_data': [{'fields': ['url'],
                 'values': [['https://www.ibm.com/cloud/machine-learning'],
                            ['https://en.wikipedia.org/wiki/IBM']]}]}


In [159]:
# if using UI for testing, use the following to get right format
import json
json.dumps(job_payload)

'{"input_data": [{"fields": ["url"], "values": [["https://www.ibm.com/cloud/machine-learning"], ["https://en.wikipedia.org/wiki/IBM"]]}]}'

In [160]:
job = client.deployments.create_job(deployment_uid, meta_props=job_payload)

In [179]:
import time

job_id = client.deployments.get_job_uid(job)

elapsed_time = 0
while client.deployments.get_job_status(job_id).get('state') != 'completed' and elapsed_time < 300:
    elapsed_time += 10
    time.sleep(10)
if client.deployments.get_job_status(job_id).get('state') == 'completed':
    job_details = client.deployments.get_job_details(job_id)
    output = job_details['entity']["scoring"]['predictions'][0]["values"]
    print(f"Output: {len(output)}")
else:
    print("Job hasn't completed successfully in 5 minutes.")

Output: 2


### check if a batch function could be deployed as online

In [182]:
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "Web scraping python function deployment batch as online",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    #client.deployments.ConfigurationMetaNames.BATCH: {},
    #client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name": "S", "num_nodes": 1}
}

In [183]:
deployment_details = client.deployments.create(function_uid, meta_props=deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: 'ca137c4a-2817-484d-a0ff-2a604cf4f6c9' started

#######################################################################################


initializing...
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='fa29422f-6b6e-44a9-92ce-1197aba6332a'
------------------------------------------------------------------------------------------------




In [185]:
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment uid = {}'.format(deployment_uid))

Deployment uid = fa29422f-6b6e-44a9-92ce-1197aba6332a


In [187]:
job_details = client.deployments.score("fa29422f-6b6e-44a9-92ce-1197aba6332a", job_payload)
pprint(job_details['predictions'][0]['values'][0][:10])

['01',
 '13',
 '2020',
 '2021',
 '37',
 'aberdeen',
 'about',
 'access',
 'across',
 'ai']


In [188]:
pprint(job_details['predictions'][0]['values'][1][:10])

['000', '05', '10', '100', '100th', '101', '102', '103', '104', '105']


In [194]:
job_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': ['url'],
        'values': [
            ['https://www.ibm.com/cloud/machine-learning']
        ]
    }]
}
pprint(job_payload)

{'input_data': [{'fields': ['url'],
                 'values': [['https://www.ibm.com/cloud/machine-learning']]}]}


In [199]:
job_details = client.deployments.score("fa29422f-6b6e-44a9-92ce-1197aba6332a", job_payload)
pprint(job_details['predictions'][0]['values'][0][:10])

['01',
 '13',
 '2020',
 '2021',
 '37',
 'aberdeen',
 'about',
 'access',
 'across',
 'ai']


- So a function that can only handle one entry at a time could only be deployed as online, batch wont work.  
- If a function can handle entries as batch, then both online and batch should both work. Just need to pay attention to the format of input and output.

## The following code shows how to make use of custom code in .py files

In [75]:
pip install git+https://github.com/CatherineCao2016/custom_lib.git

  Cloning https://github.com/CatherineCao2016/custom_lib.git to /tmp/wsuser/pip-req-build-9b97vzwk
  Running command git clone -q https://github.com/CatherineCao2016/custom_lib.git /tmp/wsuser/pip-req-build-9b97vzwk
  Created wheel for custom-lib: filename=custom_lib-0.1-py3-none-any.whl size=34559 sha256=d87b2a9f831ca793c15b58dd8c6967d32bf5f7b311caeb7df6337da8fef6f0f3
  Stored in directory: /tmp/wsuser/pip-ephem-wheel-cache-gsa5ch35/wheels/d8/0e/ba/fc319ff502868e556965f335d34f08460393e39025aee3b0b8
Successfully built custom-lib
Note: you may need to restart the kernel to use updated packages.


In [92]:
from custom_lib import sample_class

In [94]:
emp = sample_class.Employee("Y", "C", "F")

In [95]:
emp.fullname()

'Y C'

In [201]:
# adding to deployment

In [96]:
# Function Metadata.
meta_props = {
    client.repository.FunctionMetaNames.NAME: "python function from custom lib",
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID:client.software_specifications.get_id_by_name("default_py3.7"),
}

In [130]:
def py_funct(params=py_params):  
    try:
        # Import the subprocess module.
        import subprocess
        # Install required packages.
        subprocess.check_output('wget https://raw.githubusercontent.com/CatherineCao2016/custom_lib/main/custom_lib/sample_class.py', stderr=subprocess.STDOUT, shell=True)
    except subprocess.CalledProcessError as e:        
        install_err = 'subprocess.CalledProcessError:\n\n' + 'cmd:\n' + e.cmd + '\n\noutput:\n' + e.output.decode()
        raise Exception( 'Installation failed:\n' + install_err )
    
    def score(payload):
        try:
            # Import required modules.
            import sample_class

            emp = sample_class.Employee(payload['input_data'][0]['values'][0], payload['input_data'][0]['values'][1], payload['input_data'][0]['values'][2])
            
            return {"predictions": [{"fields": ["output"], "values": [emp.fullname()]}]}
        except Exception as e:
            return {"predictions": [{"error" : repr(e)}]}
        
    return score

In [131]:
#Create the function artifact.
function_artifact = client.repository.store_function(meta_props=meta_props, function=py_funct)
function_uid = client.repository.get_function_uid(function_artifact)
print("Function UID = " + function_uid)

Function UID = 50872277-bfb6-4c6e-832b-36c9e431cc14


In [156]:
client.deployments.delete("e36bbb2f-7227-4c20-9cf2-07f4b788fc02")

'SUCCESS'

In [139]:
# Deployment metadata.
deploy_meta = {
    client.deployments.ConfigurationMetaNames.NAME: "python function from custom lib deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [140]:
deployment_details = client.deployments.create(function_uid, meta_props=deploy_meta)



#######################################################################################

Synchronous deployment creation for uid: '50872277-bfb6-4c6e-832b-36c9e431cc14' started

#######################################################################################


initializing..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='07df5f1e-f730-4901-ab8a-d98737773c20'
------------------------------------------------------------------------------------------------




In [141]:
deployment_details

{'entity': {'asset': {'id': '50872277-bfb6-4c6e-832b-36c9e431cc14'},
  'custom': {},
  'deployed_asset_type': 'function',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'XXS', 'num_nodes': 1},
  'name': 'python function from custom lib deployment',
  'online': {},
  'space_id': '27f3466c-e5b6-4e88-bd72-6d6570f6d255',
  'status': {'online_url': {'url': 'https://internal-nginx-svc:12443/ml/v4/deployments/07df5f1e-f730-4901-ab8a-d98737773c20/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2021-08-10T17:03:07.126Z',
  'id': '07df5f1e-f730-4901-ab8a-d98737773c20',
  'modified_at': '2021-08-10T17:03:07.126Z',
  'name': 'python function from custom lib deployment',
  'owner': '1000331019',
  'space_id': '27f3466c-e5b6-4e88-bd72-6d6570f6d255'}}

In [142]:
# Deployment UID.
deployment_uid = client.deployments.get_uid(deployment_details)
print('Deployment uid = {}'.format(deployment_uid))

Deployment uid = 07df5f1e-f730-4901-ab8a-d98737773c20


In [101]:
job_payload = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [{
        'fields': ['employee'],
        'values': ["Catherine", "Cao", "F"
            
        ]
    }]
}
pprint(job_payload)

{'input_data': [{'fields': ['employee'], 'values': ['Catherine', 'Cao', 'F']}]}


In [144]:
# Perform prediction and display the result.
job_details = client.deployments.score(deployment_uid, job_payload)
pprint(job_details['predictions'][0]['values'])

['Catherine Cao']


# clear playground

In [212]:
client.deployments.list()

------------------------------------  -------------------------------------------------------  -----  ------------------------
GUID                                  NAME                                                     STATE  CREATED
fa29422f-6b6e-44a9-92ce-1197aba6332a  Web scraping python function deployment batch as online  ready  2021-08-10T21:59:59.832Z
4b9d1226-d0e9-4c14-abfe-f5de97d409eb  Web scraping python function deployment batch            ready  2021-08-10T21:53:22.516Z
8e5ff083-2042-4b41-b7e6-22d29593ab3a  UI created batch job                                     ready  2021-08-10T21:47:25.806Z
a9908a51-8f09-4468-b55a-b965a13454f9  Web scraping python function deployment batch            ready  2021-08-10T21:39:37.395Z
7557268a-1eb2-4bda-9217-a6cdb445b677  Web scraping python function deployment batch            ready  2021-08-10T20:49:48.107Z
0f632456-01b9-4afd-ac33-55e2fd711186  Web scraping python function deployment online           ready  2021-08-10T20:43:47.240Z
f

In [213]:
for i in ["7557268a-1eb2-4bda-9217-a6cdb445b677", "ec9b33ef-8938-41a6-b3a9-d0fa7cdf02b8" ]:
    client.deployments.delete(i)

In [214]:
client.repository.list_functions()

------------------------------------  ----------------------------------------------  ------------------------  ------
GUID                                  NAME                                            CREATED                   TYPE
ca137c4a-2817-484d-a0ff-2a604cf4f6c9  Web scraping python function updated for batch  2021-08-10T21:53:14.002Z  python
89b6e63d-dd16-4dbf-a806-813bdef5cf15  Web scraping python function updated for batch  2021-08-10T21:38:43.002Z  python
54d5b041-a0f9-44c8-886d-9865b5bc29b4  Web scraping python function demo               2021-08-10T19:48:59.002Z  python
50872277-bfb6-4c6e-832b-36c9e431cc14  python function from custom lib                 2021-08-10T17:01:54.002Z  python
f4e11908-cbc4-4796-bdc0-439c84e74300  Web scraping python function                    2021-08-10T16:08:09.002Z  python
------------------------------------  ----------------------------------------------  ------------------------  ------


In [ ]:
#client.repository.delete("")

In [ ]:
https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/analyze-data/deploy-batch-details.html